In [66]:
import pandas as pd
df_s1 = pd.read_csv('output_s1.csv')
df_s1.rename(columns={'Chat': 'channel_name'}, inplace=True)
df_s2 = pd.read_csv('output_s2.csv')
columns_identical = df_s1.columns.equals(df_s2.columns)
print(f"are columns in output_s1 und output_s2 identical? {columns_identical}")
df_s1.to_csv('output_s1.csv', index=False)


Sind die Spaltennamen in output_s1 und output_s2 identisch? True


In [67]:
import pandas as pd
df = pd.read_csv("output_s1.csv")
df_s1 = df_s1[df_s1['channel_name'] == 'die_basis_funkt']
print(df_s1.columns.tolist())
print("Number of rows in df_s1:", len(df_s1))

['channel_name', 'link_to_message', 'could_be_displayed', 'user', 'user_link', 'user_img', 'text', 'nb_hashtags', 'hashtags', 'is_reply_img', 'is_reply_user', 'is_reply_text', 'stickers', 'nb_photos', 'photos', 'nb_videos', 'videos', 'videos_times', 'nb_links', 'links', 'link_img', 'link_site', 'link_title', 'link_description', 'views', 'datetime', 'edited']
Number of rows in df_s1: 1783


so this is key here below, this is the list of linked chats without threshold in chat dieBasis

In [71]:
print(df_s1.columns.tolist())
#to check if df_s1 only has the correct channel data
unique_values = df_s1['channel_name'].unique()
for value in unique_values:
    print(value)

['channel_name', 'link_to_message', 'could_be_displayed', 'user', 'user_link', 'user_img', 'text', 'nb_hashtags', 'hashtags', 'is_reply_img', 'is_reply_user', 'is_reply_text', 'stickers', 'nb_photos', 'photos', 'nb_videos', 'videos', 'videos_times', 'nb_links', 'links', 'link_img', 'link_site', 'link_title', 'link_description', 'views', 'datetime', 'edited']
die_basis_funkt


In [72]:
import re
telegram_chat_pattern = r't\.me\/([\w-]+)'

telegram_chat_links = df_s1[df_s1['links'].str.contains(telegram_chat_pattern, na=False)]['links']
telegram_chat_links = telegram_chat_links.str.extractall(telegram_chat_pattern).reset_index(drop=True)[0].unique()
output_file = 'seed2.txt'
with open(output_file, 'w') as file:
    for link in telegram_chat_links:
        file.write(link + '\n')
    

/tmp/ipykernel_458/729679295.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  telegram_chat_links = df_s1[df_s1['links'].str.contains(telegram_chat_pattern, na=False)]['links']


In [74]:
# lets now only include in output from second seed 'seed2.txt' those chats that were from die_basis_funkt
# no threshold, one single mention is enough
import pandas as pd

# Step 1: Read the list of channels from seed2.txt
with open("seed2.txt", "r") as file:
    channels_list = file.read().splitlines()

# Step 2: Filter df_s2 to include only the rows where 'channel_name' matches the list
df_s2_v2 = df_s2[df_s2['channel_name'].isin(channels_list)]

# Optional: Verify the filtering by checking if all 'channel_name' in df_s2_v2 are in the channels_list
# This should return True if the filtering is correct
verification_result = df_s2_v2['channel_name'].isin(channels_list).all()

print(f"Verification that all channel names in df_s2_v2 are in the list: {verification_result}")
print(df_s2_v2.columns.tolist())
print("Number of rows in df_s2_v2:", len(df_s2_v2))
print("Number of rows in df_s2:", len(df_s2))



Verification that all channel names in df_s2_v2 are in the list: True
['channel_name', 'link_to_message', 'could_be_displayed', 'user', 'user_link', 'user_img', 'text', 'nb_hashtags', 'hashtags', 'is_reply_img', 'is_reply_user', 'is_reply_text', 'stickers', 'nb_photos', 'photos', 'nb_videos', 'videos', 'videos_times', 'nb_links', 'links', 'link_img', 'link_site', 'link_title', 'link_description', 'views', 'datetime', 'edited']
Number of rows in df_s2_v2: 70908
Number of rows in df_s2: 125599


In [82]:
df_s2_v2.to_csv("df_s2_v2.csv", index=False)

print("df_s2_v2.csv has been saved")

df_s2_v2.csv has been saved to


In [75]:
# now export the chat links from df_s2_v2 to seed3.txt; this needs to be rewritten to only include those mentioned more than once.
import re
telegram_chat_pattern = r't\.me\/([\w-]+)'

telegram_chat_links = df_s2_v2[df_s2_v2['links'].str.contains(telegram_chat_pattern, na=False)]['links']
telegram_chat_links = telegram_chat_links.str.extractall(telegram_chat_pattern).reset_index(drop=True)[0].unique()
output_file = 'seed3.txt'
with open(output_file, 'w') as file:
    for link in telegram_chat_links:
        file.write(link + '\n')

/tmp/ipykernel_458/4217214506.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  telegram_chat_links = df_s2_v2[df_s2_v2['links'].str.contains(telegram_chat_pattern, na=False)]['links']


In [79]:
### inspect how many times links were forwarded
telegram_chat_pattern = r't\.me\/([\w-]+)'
telegram_chat_links = df_s2_v2[df_s2_v2['links'].str.contains(telegram_chat_pattern, na=False)]['links']
telegram_chat_links = telegram_chat_links.str.extractall(telegram_chat_pattern).reset_index(drop=True)[0]

# Häufigkeit jedes Links zählen
link_frequency = telegram_chat_links.value_counts()

# Categorize each frequency into specified bins
bins = [-1, 1, 5, 10, 20, float('inf')]  # Define bins edges
labels = ['1', '2-5', '6-10', '11-20', '>20']  # Labels for the bins
categorized = pd.cut(link_frequency, bins=bins, labels=labels)

# Count occurrences in each bin for absolute numbers
distribution_abs = categorized.value_counts().sort_index()

# Calculate relative frequency (percentage) of each bin
distribution_rel = (distribution_abs / distribution_abs.sum()) * 100

# Create a DataFrame to neatly display both absolute and relative frequencies
distribution_table = pd.DataFrame({'Absolute Frequency': distribution_abs, 'Relative Frequency (%)': distribution_rel})

print(distribution_table)

/tmp/ipykernel_458/708043668.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  telegram_chat_links = df_s2_v2[df_s2_v2['links'].str.contains(telegram_chat_pattern, na=False)]['links']


       Absolute Frequency  Relative Frequency (%)
count                                            
1                    1219               45.031400
2-5                   853               31.510898
6-10                  240                8.865903
11-20                 164                6.058367
>20                   231                8.533432


In [81]:
### now actually create seed3.txt with threshold 2
# Filter out links with frequency >= 2
links_to_save = link_frequency[link_frequency >= 2].index

# Convert the filtered links to a string with each link on a new line
links_str = "\n".join(links_to_save)

# Write the filtered links to seed3.txt
with open("seed3.txt", "w") as file:
    file.write(links_str)

print(f"Saved {len(links_to_save)} links to seed3.txt.")

Saved 1488 links to seed3.txt.


here I calculate how big the threshold for n-links should be for each seed. I decide in teh end that it is n=0 for the second seed and n=2 for the third seed, I think.

In [32]:
###wie hoch soll der threshold sein für n-links 
import pandas as pd
import re

# Dein vorhandener Code, um Telegram-Links zu extrahieren
telegram_chat_pattern = r'(t\.me\/\w+)'
telegram_chat_links = df_s1[df_s1['links'].str.contains(telegram_chat_pattern, na=False)]['links']
telegram_chat_links = telegram_chat_links.str.extractall(telegram_chat_pattern).reset_index(drop=True)[0]

# Häufigkeit jedes Links zählen
link_frequency = telegram_chat_links.value_counts()

# Ergebnisse für verschiedene Thresholds speichern
results = {}

# Thresholds testen von 2 bis 10
for threshold in range(2, 11):
    # Anzahl der Chats, die den Threshold erfüllen
    filtered_chats_count = link_frequency[link_frequency >= threshold].count()
    results[threshold] = filtered_chats_count

# Ergebnisse ausgeben
print("Chats pro Threshold:")
for threshold, count in results.items():
    print(f"Threshold {threshold}: {count} Chats")

Chats pro Threshold:
Threshold 2: 56 Chats
Threshold 3: 42 Chats
Threshold 4: 33 Chats
Threshold 5: 28 Chats
Threshold 6: 21 Chats
Threshold 7: 16 Chats
Threshold 8: 13 Chats
Threshold 9: 10 Chats
Threshold 10: 10 Chats


/tmp/ipykernel_458/2698969296.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  telegram_chat_links = df_s1[df_s1['links'].str.contains(telegram_chat_pattern, na=False)]['links']


In [56]:
### berechnen wie viele Chats je nach threshold noch mit drinnen wären
import pandas as pd
import re

# Annahme: df_s1 und df_s2 sind bereits geladen

# Schritt 1: Anpassen der Extraktion von Telegram-Links aus df_s1
telegram_chat_pattern = r't\.me\/([\w-]+)'
telegram_links_s1 = df_s1['links'].str.extractall(telegram_chat_pattern).reset_index(drop=True)[0]
link_frequency_s1 = telegram_links_s1.value_counts()

# Bereite die Ergebnisse für Seed2 basierend auf verschiedenen Thresholds vor
s2_chats_per_threshold = {}
thresholds = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
for threshold in thresholds:
    filtered_chats = link_frequency_s1[link_frequency_s1 >= threshold].index.tolist()
    s2_chats_per_threshold[threshold] = filtered_chats

# Schritt 3: Analysiere Seed3 basierend auf Seed2 für jeden Threshold
results_s3 = {}
df_s2 = pd.read_csv('output_s2.csv')  # Lade die Daten aus output_s2.csv

for threshold_s2 in thresholds:
    s2_chats = s2_chats_per_threshold[threshold_s2]
    df_filtered = df_s2[df_s2['channel_name'].isin(s2_chats)]
    if not df_filtered.empty:
        telegram_links_s2 = df_filtered['links'].str.extractall(telegram_chat_pattern).reset_index(drop=True)[0]
        link_frequency_s2 = telegram_links_s2.value_counts()

        for threshold_s3 in thresholds:
            filtered_chats_count = link_frequency_s2[link_frequency_s2 >= threshold_s3].count()
            results_s3[(threshold_s2, threshold_s3)] = filtered_chats_count
    else:
        print(f"Keine Ergebnisse für Threshold {threshold_s2}")

# Ergebnisse ausgeben
print("Chats in S3 basierend auf Thresholds in S2:")
for (threshold_s2, threshold_s3), count in results_s3.items():
    print(f"Seed2 Threshold {threshold_s2}, Seed3 Threshold {threshold_s3}: {count} Chats")


Chats in S3 basierend auf Thresholds in S2:
Seed2 Threshold 0, Seed3 Threshold 0: 2707 Chats
Seed2 Threshold 0, Seed3 Threshold 2: 1488 Chats
Seed2 Threshold 0, Seed3 Threshold 4: 874 Chats
Seed2 Threshold 0, Seed3 Threshold 6: 635 Chats
Seed2 Threshold 0, Seed3 Threshold 8: 525 Chats
Seed2 Threshold 0, Seed3 Threshold 10: 431 Chats
Seed2 Threshold 0, Seed3 Threshold 12: 364 Chats
Seed2 Threshold 0, Seed3 Threshold 14: 317 Chats
Seed2 Threshold 0, Seed3 Threshold 16: 288 Chats
Seed2 Threshold 0, Seed3 Threshold 18: 261 Chats
Seed2 Threshold 0, Seed3 Threshold 20: 239 Chats
Seed2 Threshold 2, Seed3 Threshold 0: 1352 Chats
Seed2 Threshold 2, Seed3 Threshold 2: 745 Chats
Seed2 Threshold 2, Seed3 Threshold 4: 447 Chats
Seed2 Threshold 2, Seed3 Threshold 6: 330 Chats
Seed2 Threshold 2, Seed3 Threshold 8: 256 Chats
Seed2 Threshold 2, Seed3 Threshold 10: 217 Chats
Seed2 Threshold 2, Seed3 Threshold 12: 178 Chats
Seed2 Threshold 2, Seed3 Threshold 14: 151 Chats
Seed2 Threshold 2, Seed3 Thresho